Обратил внимание, что задание выглядит немного по-другому уже после того как всё сделал (код был практически готов уже через 3 дня после объявления о том, что будет домашнее задание). Менять некогда, поэтому оставлю как есть.

# CIAN.ru

In [10]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from functools import reduce
from sklearn import preprocessing
import pandas as pd
import numpy as np
import random

In [2]:
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))

In [3]:
page = 1

In [3]:
#кусочек html циана с регионами
distlist_html = open('cian_links.txt')
distlist = []
for line in distlist_html:
    try:
        distlist.append(re.findall('"http://.*"', line)[0].rstrip('"').lstrip('"'))
    except:
        pass

In [4]:
distlist

['http://www.cian.ru/kupit-kvartiru-moskva-akademicheskiy-04100/',
 'http://www.cian.ru/kupit-kvartiru-moskva-vyhino-zhulebino-0472/',
 'http://www.cian.ru/kupit-kvartiru-moskva-kryukovo-04154/',
 'http://www.cian.ru/kupit-kvartiru-moskva-nagatinskiy-zaton-0492/',
 'http://www.cian.ru/kupit-kvartiru-moskva-ryazanskiy-0481/',
 'http://www.cian.ru/kupit-kvartiru-moskva-troparevo-nikulino-04122/',
 'http://www.cian.ru/kupit-kvartiru-moskva-alekseevskiy-0439/',
 'http://www.cian.ru/kupit-kvartiru-moskva-gagarinskiy-04101/',
 'http://www.cian.ru/kupit-kvartiru-moskva-kuzminki-0474/',
 'http://www.cian.ru/kupit-kvartiru-moskva-nagornyy-0493/',
 'http://www.cian.ru/kupit-kvartiru-moskva-savelovskiy-0434/',
 'http://www.cian.ru/kupit-kvartiru-moskva-filevskiy-park-04123/',
 'http://www.cian.ru/kupit-kvartiru-moskva-altufevskiy-0440/',
 'http://www.cian.ru/kupit-kvartiru-moskva-golovinskiy-0428/',
 'http://www.cian.ru/kupit-kvartiru-moskva-kuncevo-04115/',
 'http://www.cian.ru/kupit-kvartiru-mo

In [5]:
len(distlist)

125

### Для ЦАО находим главную ссылку и в ней находим **&p=1&**, это и есть переход по страницам

In [4]:
district = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [5]:
district

'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

### Гуляем по страницам и качаем с них все ссылки на наши квартирки

In [13]:
links = []
sample = 5
for i,district in enumerate(distlist):
    print (i)
    for page in random.sample(list(range(0, 30)),k=sample):
        page_url =  district.format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')
        print("Parsing page %i" % page)
        try:
            flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
            flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))
        except:
            print("Caught exception on page %i" % page)
        for link in flat_urls:
            if link.isdigit():
                links.append(link)

0
Parsing page 29
Parsing page 27
Parsing page 13
Parsing page 5
Parsing page 3
1
Parsing page 18
Parsing page 2
Parsing page 7
Parsing page 19
Parsing page 26
2
Parsing page 20
Parsing page 12
Parsing page 23
Parsing page 19
Parsing page 5
3
Parsing page 4
Parsing page 22
Parsing page 28
Parsing page 17
Parsing page 18
4
Parsing page 28
Parsing page 27
Parsing page 16
Parsing page 29
Parsing page 7
5
Parsing page 4
Parsing page 7
Parsing page 24
Parsing page 13
Parsing page 11
6
Parsing page 14
Parsing page 15
Parsing page 4
Parsing page 5
Parsing page 26
7
Parsing page 26
Parsing page 9
Parsing page 22
Parsing page 19
Parsing page 14
8
Parsing page 10
Parsing page 6
Parsing page 0
Parsing page 19
Parsing page 26
9
Parsing page 22
Parsing page 25
Parsing page 20
Parsing page 6
Parsing page 21
10
Parsing page 26
Parsing page 29
Parsing page 7
Parsing page 4
Parsing page 8
11
Parsing page 20
Parsing page 8
Parsing page 23
Parsing page 6
Parsing page 29
12
Parsing page 23
Parsing page 15

In [14]:
len(links)

16850

In [15]:
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)

In [16]:
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon

In [17]:
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    return room_number




Все try-except в коде - это не затычка, чтобы прикрыть проблемы кода, неспособного обработать страницу, а часть своеобразной логики парсера

In [18]:
def getFloor (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Этаж:', table)
    if len(t)<=1:
        return None
    try:
        if (t[1].find(re.findall("\d+",t[1])[0]) < t[1].find("–")) or (t[1].find("–")==-1):   
            return (int(re.findall("\d+",t[1])[0]), int(re.findall("\d+",t[1])[1]))
        else:
            return None
    except:
        #findall не нашел нужного, поэтому re.findall("\d+",t[1])[0  ИЛИ  1] вернул ошибку
        return None

In [19]:
def getHtype (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Тип дома:', table)
    #проверка, есть ли "Тип дома:" в table:
    if (len(t)>=2):
        t = re.split("\n",t[1])
        b = []
        count = 0
        while not(":" in t[count]):
            try:
                if re.findall("\w+[-\w]*",t[count]):
                    b.append(re.findall("\w+[-\w]*",t[count])[0])
            except:
                # не знаю, нужен ли в этом случае try-except, но всё работает, и, возможно, 
                # когда-то у меня были идеи, по которым я поставил его здесь,
                # поэтому трогать не буду
                pass
            count += 1
        return ",".join(b)
    else:
        return None

In [20]:
def getOffertype (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Тип продажи:', table)
    if (len(t)>=2):
        t = re.split("\n",t[1])
        b = []
        count = 0
        while not(":" in t[count]):
            try:
                if re.findall("\w+",t[count]):
                    b.append(re.findall("\w+",t[count])[0])
            except:
                #то же, что и в предыдущем
                pass
            count += 1
        return ",".join(b)
    else:
        return None

In [21]:
def getOverallArea (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Общая площадь:', table)
    if(len(t)<=1):
        return None
    if (t[1].find(re.findall("\d+",t[1])[0]) < t[1].find("–")) or (t[1].find("–")==-1):   
        try:
            #тут я в определенный момент нашёл ошибку, и мне захотелось сделать всё так, чтобы как можно меньше
            #кода изменить
            #
            #забираем полностью первую попавщуюся float с запятой или int, и пытаемся скастить её во float:
            return float(re.findall("(?<!\d)(?<!\d,)\d+(?!\d)(?!,\d)|\d+,\d+",t[1])[0])
        except:
            #если не удается, значит, первым нам попался float с запятой, и тогда мы его кастим по-нормальному:
            return float(".".join(re.findall("\d+,\d*",t[1])[0].split(",")))
    else:
        return None

In [22]:
def getRoomArea (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Площадь комнат:', table)
    if(len(t)<=1):
        return None
    try:
        #жуткий код. Я не помню, как это работает, но оно, кажется, работает
        #забирает площади всех указанных в объявлении комнат, которые в объявлении указаны в формате 12,1-12,1-12,1
        #кроме площади комнат,в таком формате больше ничего не бывает, поэтому работает
        #правильно обработает и 12,1, и 121
        if (t[1].find(re.findall("\d+",t[1])[0]) < t[1].find("–")) or (t[1].find("–")==-1):   
            return tuple(map( (lambda x: float(x.replace(",","."))), re.findall('[\d+,?\d*-]*\d+,?\d*',t[1])[0].split("-")))
        else:
            return None
    except:
        return None

In [23]:
def getLivingArea (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Жилая площадь:', table)
    if(len(t)<=1):
        return None
    try:
        if (t[1].find(re.findall("\d+",t[1])[0]) < t[1].find("–")) or (t[1].find("–")==-1):   
            try:
                #аналогично getOverallArea
                return float(re.findall("(?<!\d)(?<!\d,)\d+(?!\d)(?!,\d)|\d+,\d+",t[1])[0])
            except:
                return float(".".join(re.findall("\d+,\d*",t[1])[0].split(",")))
        else:
            return None
    except:
        return None

In [24]:
def getKitArea (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Площадь кухни:', table)
    if(len(t)<=1):
        return None
    try:
        if (t[1].find(re.findall("\d+",t[1])[0]) < t[1].find("–")) or (t[1].find("–")==-1):   
            try:
                #аналогично getOverallArea
                return float(re.findall("(?<!\d)(?<!\d,)\d+(?!\d)(?!,\d)|\d+,\d+",t[1])[0])
            except:
                return float(".".join(re.findall("\d+,\d*",t[1])[0].split(",")))
        else:
            return None
    except:
        return None

In [25]:
def getSLavNum (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Раздельных санузлов:', table)
    if(len(t)<=1):
        return None
    #как обычно, проверяем, что прочерк не идёт первым:
    if (t[1].find(re.findall("\d+",t[1])[0]) < t[1].find("–")) or (t[1].find("–")==-1):   
        return int(re.findall("\d+",t[1])[0])
    else:
        return None

In [26]:
def getULavNum (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Совмещенных санузлов:', table)
    if(len(t)<=1):
        return None
    #как обычно, проверяем, что прочерк не идёт первым:
    if (t[1].find(re.findall("\d+",t[1])[0]) < t[1].find("–")) or (t[1].find("–")==-1):   
        return int(re.findall("\d+",t[1])[0])
    else:
        return None

Посмотреть возможные типы балконов!

In [27]:
balset = {"балк","лодж"}
elevset = {"пасс","груз"}

In [28]:
def getBal (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Балкон:', table)
    if (len(t)>=2):
        #а здесь вообще жуть...
        #на циане типы и количество указаны через перенос строки
        try:
            if (t[1].find(re.findall("нет",t[1])[0]) < t[1].find("–")) or (t[1].find("–")==-1):   
                return ["нет"]
        except:
            pass
        t = re.split("\n",t[1])
        b = []
        count = 0
        #считываем с каждой строки, пока не наткнёмся на следующую категорию в таблице
        while not(":" in t[count]):
            try:
                if re.findall("\d+ \w+",t[count]):
                    b.append(re.findall("\d+ \w+",t[count])[0])
            except:
                pass
            count += 1
        return tuple(b)
    else:
        return None

In [29]:
def getElev (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Лифт:', table)
    if (len(t)>=2):
        t = re.split("\n",t[1])
        b = []
        count = 0
        #приблизительно то же, что и в прошлом
        while not(":" in t[count]):
            try:
                if re.findall("\d+ \w+",t[count]):
                    b = reduce((lambda x,y: x+y), b, re.findall("\+*\d+ \w+",t[count]))
            except:
                pass
            count += 1
        return tuple(b)
    else:
        return None

In [30]:
def getParktype (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Парковка:', table)
    if (len(t)>=2):
        t = re.split("\n",t[1])
        b = []
        count = 0
        #аналогично
        while not(":" in t[count]):
            try:
                if re.findall("\w+",t[count]):
                    b.append(re.findall("\w+",t[count])[0])
            except:
                pass
            count += 1
        return ",".join(b)
    else:
        return None

In [31]:
def getView (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Вид из окна:', table)
    if (len(t)>=2):
        t = re.split("\n",t[1])
        b = ""
        count = 0
        #аналогично
        while not(":" in t[count]):
            try:
                if re.findall("\w+",t[count]):
                    b = reduce((lambda x,y: x+y),re.findall("\w+",t[count]),b)
            except:
                pass
            count += 1
            if count == len(t):
                break
        return "".join(b)
    else:
        return None

In [32]:
def getRdate (flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    t = re.split('Сдача ГК:', table)
    if (len(t)>=2):
        t = re.split("\n",t[1])
        b = ""
        count = 0
        #2 формата на циане: год или квартал + год
        while True:
            if re.findall('дом сдан', t[count]):
                return (0,0)
            try:
                if re.findall("\d+ кв. \d+",t[count]):
                    b = re.findall("\d+ кв. \d+",t[count])[0]
                    b = (int(re.findall("\d+ ", b)[0]),int(re.findall(" \d+", b)[0]))
                    break
                if re.findall("\d+ год",t[count]):
                    b = re.findall("\d+ год",t[count])[0]
                    #если квартал не указан, он нулевой (нулевого квартала не бывает, поэтому 0 можно заменить после)
                    b = (0,int(re.findall("\d+", b)[0]))
                    break
                if '—' in t:
                    return None
            except:
                pass
            count += 1
        return b
    else:
        return None

In [33]:
def getFlatDicts (links):
    flats = []
    count = 1
    for link in links:
        #иногда ругается, и не может распарсить цену квартиры. При этом это не зависит от конкретной страницы,
        #и тогда на эту проблему можно не обращать внимание
        try:
            dic = {}
            flat_url = 'http://www.cian.ru/sale/flat/' + str(link) + '/'
            print("Parsing page %i out of %i: %s" % (count, len(links), flat_url))
            count += 1
            flat_page = requests.get(flat_url)
            flat_page = flat_page.content
            flat_page = BeautifulSoup(flat_page, 'lxml')
            dic['price'] = getPrice(flat_page)
            dic['lat'], dic['lon'] = getCoords(flat_page)
            dic['rooms'] = getRoom(flat_page)
            dic['floor'] = getFloor(flat_page)
            dic['htype'] = getHtype(flat_page)
            dic['offertype'] = getOffertype(flat_page)
            dic['overallarea'] = getOverallArea(flat_page)
            dic['roomarea'] = getRoomArea(flat_page)
            dic['livarea'] = getLivingArea(flat_page)
            dic['kitarea'] = getKitArea(flat_page)
            dic['slavnum'] = getSLavNum(flat_page)
            dic['ulavnum'] = getULavNum(flat_page)
            dic['bal'] = getBal(flat_page)
            dic['elev'] = getElev(flat_page)
            dic['parktype'] = getParktype(flat_page)
            dic['view'] = getView(flat_page)
            dic['rdate'] = getRdate(flat_page)
            flats.append(dic.copy())
        except:
            pass
    return flats

In [34]:
firstset = getFlatDicts(random.sample(set(links), k=2000))

Parsing page 1 out of 2000: http://www.cian.ru/sale/flat/50633736/
Parsing page 2 out of 2000: http://www.cian.ru/sale/flat/150102550/
Parsing page 3 out of 2000: http://www.cian.ru/sale/flat/149231080/
Parsing page 4 out of 2000: http://www.cian.ru/sale/flat/149068448/
Parsing page 5 out of 2000: http://www.cian.ru/sale/flat/150690935/
Parsing page 6 out of 2000: http://www.cian.ru/sale/flat/148996085/
Parsing page 7 out of 2000: http://www.cian.ru/sale/flat/150856964/
Parsing page 8 out of 2000: http://www.cian.ru/sale/flat/150163885/
Parsing page 9 out of 2000: http://www.cian.ru/sale/flat/150978462/
Parsing page 10 out of 2000: http://www.cian.ru/sale/flat/150933465/
Parsing page 11 out of 2000: http://www.cian.ru/sale/flat/150813428/
Parsing page 12 out of 2000: http://www.cian.ru/sale/flat/150151953/
Parsing page 13 out of 2000: http://www.cian.ru/sale/flat/151073165/
Parsing page 14 out of 2000: http://www.cian.ru/sale/flat/142821939/
Parsing page 15 out of 2000: http://www.cian

In [35]:
firstset_backup = []
for i in range(len(firstset)):
    firstset_backup.append(firstset[i].copy())

In [36]:
#обработка балконов, лифтов, комнат
def splittuples (l):
    maxel = 0
    maxroom = 0
    maxbal = 0
    for dic in l:
        boofer = dic['floor']
        dic['floor'] = boofer[0]
        dic['hfloor'] = boofer[1]
        try:
            maxel = max(len(dic['elev']), maxel)
        except:
            pass
        try:
            maxroom = max(len(dic['roomarea']), maxroom)
        except:
            pass
        try:
            maxbal = max(len(dic['bal']), maxbal)
        except:
            pass
    for dic in l:
        boofer = dic['elev']
        dic.pop('elev',None)
        for i in range(maxel):
            try:
                dic['elev'+str(i)] = boofer[i]
            except:
                dic['elev'+str(i)] = None
        boofer = dic['roomarea']
        dic.pop('roomarea',None)
        for i in range(maxroom):
            try:
                dic['roomarea'+str(i)] = boofer[i]
            except:
                try:
                    if int(dic['rooms'])-1 < i:
                        dic['roomarea'+str(i)] = 0
                    else:
                        dic['roomarea'+str(i)] = None
                except:
                    dic['roomarea'+str(i)] = None
        boofer = dic['bal']
        dic.pop('bal',None)
        for i in range(maxroom):
            try:
                dic['bal'+str(i)] = boofer[i]
            except:
                dic['bal'+str(i)] = None

In [43]:
firstset = []
for i in range(len(firstset_backup)):
    firstset.append(firstset_backup[i].copy())

In [37]:
splittuples(firstset)

In [38]:
firstset[0]

{'bal0': '1 лодж',
 'bal1': None,
 'bal2': None,
 'bal3': None,
 'bal4': None,
 'bal5': None,
 'bal6': None,
 'elev0': '2 груз',
 'elev1': None,
 'floor': 12,
 'hfloor': 13,
 'htype': 'новостройка,монолитный',
 'kitarea': 5.0,
 'lat': 55.790647,
 'livarea': 35.2,
 'lon': 37.524314,
 'offertype': 'свободная',
 'overallarea': 46.2,
 'parktype': None,
 'price': 7810192,
 'rdate': None,
 'roomarea0': 22.0,
 'roomarea1': None,
 'roomarea2': 0,
 'roomarea3': 0,
 'roomarea4': 0,
 'roomarea5': 0,
 'roomarea6': 0,
 'rooms': '2',
 'slavnum': None,
 'ulavnum': 1,
 'view': 'двор'}

In [39]:
data = pd.DataFrame(firstset)

In [40]:
data.to_csv('data.csv')

In [48]:
data.drop_duplicates(inplace=True)

In [50]:
len(data)

848

Дальше, мне кажется, заглядывать не нужно. Я не помню, зачем этот код нужен XD

In [751]:
def to_dummies (diclist, key):
    s0 = set()
    for dic in diclist:
        s0.add(dic[key])
    s0 = list(s0)
    for dic in diclist:
        i = s0.index(dic[key])
        if i!=0:
            dic[key] = [0]*(i-1)+[1]+[0]*(len(s0)-i)
        else:
            dic[key] = [1]+[0]*(len(s0)-1)

In [797]:
dummyset = {x for x in firstset[0].keys() if ("parktype" in x or "elev" in x or "bal" in x or "view" in x or "offertype" in x or "htype" in x)}

In [798]:
dummyset

{'bal0',
 'bal1',
 'bal2',
 'bal3',
 'bal4',
 'bal5',
 'bal6',
 'elev0',
 'elev1',
 'htype',
 'offertype',
 'parktype',
 'view'}

In [799]:
for dummy in dummyset:
    to_dummies(firstset, dummy)

In [800]:
firstset[0]

{'bal0': [0, 0, 0, 0, 0, 1, 0, 0],
 'bal1': [1, 0],
 'bal2': [1, 0],
 'bal3': [1],
 'bal4': [1],
 'bal5': [1],
 'bal6': [1],
 'elev0': [0, 0, 0, 0, 0, 1, 0, 0],
 'elev1': [0, 1, 0, 0],
 'floor': 3,
 'hfloor': 5,
 'htype': [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 'kitarea': None,
 'lat': 55.75019,
 'livarea': 146.0,
 'lon': 37.597459,
 'offertype': [0, 0, 1, 0, 0],
 'overallarea': 182.6,
 'parktype': [1, 0, 0, 0],
 'price': 147000000,
 'rdate': None,
 'roomarea0': None,
 'roomarea1': None,
 'roomarea2': None,
 'roomarea3': None,
 'roomarea4': None,
 'roomarea5': 0,
 'roomarea6': 0,
 'rooms': '5',
 'slavnum': None,
 'ulavnum': 2,
 'view': [0, 0, 1, 0]}

In [801]:
for dic in firstset:
    try:
        if dic['slavnum'] == None and dic['ulavnum'] >= 1:
            dic['slavnum'] = 0
    except:
        pass
    try:
        if dic['ulavnum'] == None and dic['slavnum'] >= 1:
            dic['ulavnum'] = 0
    except:
        pass
    if dic['rdate'] == None:
        dic['rdate'] = 0

In [802]:
cleanset = []
for dic in firstset:
    b = True
    for key in dic.keys():
        if dic[key] == None:
            b = False
            break
    if(b):
        cleanset.append(dic)

In [803]:
len(cleanset)

678

In [719]:
flat_url = 'http://www.cian.ru/sale/flat/150457338/'
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')

In [731]:
getFlatDicts(['150055757'])

Parsing page 1 out of 1: http://www.cian.ru/sale/flat/150055757/


[{'bal': 'нет',
  'elev': (),
  'floor': (2, 3),
  'htype': 'вторичка,кирпичный',
  'kitarea': 15.0,
  'lat': 55.428041,
  'livarea': 61.0,
  'lon': 37.272934,
  'offertype': 'свободная',
  'overallarea': 80.0,
  'parktype': None,
  'price': 4000000,
  'rdate': None,
  'roomarea': (20.0,),
  'rooms': '3',
  'slavnum': 1,
  'ulavnum': None,
  'view': 'двор'}]

In [718]:
flat_page = requests.get(flat_url)
flat_page = flat_page.content
flat_page = BeautifulSoup(flat_page, 'lxml')
price = flat_page.find('div', attrs={'class':'object_descr_price'})
price

<div class="object_descr_price">
            
                

3 999 000 руб.

            
            


            
        </div>

In [56]:
re.findall("(?<!\d,)\d+(?!\d*,\d)",'32,232')

['32']

In [76]:
re.findall("(?<!\d)(?<!\d,)\d+(?!\d)(?!,\d)|\d+,\d+",'Цена квартиры: 322,32\n Площадь дома: 43223')

['322,32', '43223']

In [45]:
l = []
for inst in firstset:
    s = ""
    for key in sorted(inst.keys()):
        s = s + key + str(inst[key])
    l.append(s)

In [47]:
len(set(l))

848

In [43]:
a.keys()

dict_keys(['livarea', 'bal2', 'lon', 'rdate', 'bal3', 'elev1', 'roomarea2', 'roomarea3', 'offertype', 'overallarea', 'rooms', 'bal4', 'slavnum', 'kitarea', 'roomarea0', 'view', 'htype', 'parktype', 'price', 'floor', 'hfloor', 'roomarea4', 'bal1', 'roomarea1', 'ulavnum', 'bal0', 'lat', 'elev0'])